In [1]:
#xgboost algo

In [2]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install prophet
!pip install statsmodels
!pip install scikit-learn
!pip install numpy seaborn
!pip install plotly
!pip install numpy pandas matplotlib seaborn
!pip install jovian opendatasets xgboost graphviz lightgbm scikit-learn xgboost lightgbm
!pip install --upgrade scikit-learn

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter('ignore')
import os
import plotly.express as px
import matplotlib.style as style

import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor # wrapper
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# Modelling
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Visualizations
sns.set(rc={'figure.figsize': (20, 5)})

style.use('seaborn-poster')
style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.facecolor'] = '#00000000'
%matplotlib inline

In [40]:
from prophet import Prophet

In [41]:
pd.options.display.max_columns=100

In [42]:
#Load The data
df_train = pd.read_csv('C:/Users/91773/OneDrive/Desktop/Unmesh/train.csv' , nrows=2000)
df_features = pd.read_csv('C:/Users/91773/OneDrive/Desktop/Unmesh/features.csv' ,nrows=2000)
df_stores = pd.read_csv('C:/Users/91773/OneDrive/Desktop/Unmesh/stores.csv', nrows=2000)


In [43]:
# convert Date to datetime
df_train["Date"] = pd.to_datetime(df_train["Date"])
# df_test["Date"] = pd.to_datetime(df_test["Date"])
df_features["Date"] = pd.to_datetime(df_features["Date"])

In [44]:
train_data = df_train.merge(df_features, on=['Store','Date'], how='inner').merge(df_stores, on=['Store'], how='inner')
#test_data = df_test.merge(df_features, on=['Store', 'Date'], how='inner').merge(df_stores, on=['Store'], how='inner')

In [45]:
train_data.shape

(2000, 17)

In [46]:
train_data.isna().sum()

Store              0
Dept               0
Date               0
Weekly_Sales       0
IsHoliday_x        0
Temperature        0
Fuel_Price         0
MarkDown1       1288
MarkDown2       1413
MarkDown3       1302
MarkDown4       1288
MarkDown5       1288
CPI                0
Unemployment       0
IsHoliday_y        0
Type               0
Size               0
dtype: int64

In [47]:
train_data.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y','IsHoliday_x':'IsHoliday'}, inplace=True)

In [48]:
cols_to_drop = ["IsHoliday_y","MarkDown1", "MarkDown2", "MarkDown3","MarkDown4","MarkDown5"]
train_data.drop(columns=cols_to_drop, inplace=True)

In [49]:
train_data.head()

,Store,Dept,ds,y,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,211.096358,8.106,A,151315
1,1,2,2010-02-05,50605.27,False,42.31,2.572,211.096358,8.106,A,151315
2,1,3,2010-02-05,13740.12,False,42.31,2.572,211.096358,8.106,A,151315
3,1,4,2010-02-05,39954.04,False,42.31,2.572,211.096358,8.106,A,151315
4,1,5,2010-02-05,32229.38,False,42.31,2.572,211.096358,8.106,A,151315


In [50]:
ts_xgboost = train_data.copy()
ts_xgboost

,Store,Dept,ds,y,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,211.096358,8.106,A,151315
1,1,2,2010-02-05,50605.27,False,42.31,2.572,211.096358,8.106,A,151315
2,1,3,2010-02-05,13740.12,False,42.31,2.572,211.096358,8.106,A,151315
3,1,4,2010-02-05,39954.04,False,42.31,2.572,211.096358,8.106,A,151315
4,1,5,2010-02-05,32229.38,False,42.31,2.572,211.096358,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...
1995,1,9,2012-10-26,41040.84,False,69.16,3.506,223.444251,6.573,A,151315
1996,1,10,2012-10-26,26285.27,False,69.16,3.506,223.444251,6.573,A,151315
1997,1,11,2012-10-26,22399.83,False,69.16,3.506,223.444251,6.573,A,151315
1998,1,12,2012-10-26,8658.41,False,69.16,3.506,223.444251,6.573,A,151315


In [51]:
# Converting categorical cols to numerical cols and removing old cols
mappings = {0:0, "False": 0, "True": 1}
ts_xgboost["IsHoliday_cat"] = ts_xgboost["IsHoliday"].map(mappings)
ts_xgboost = ts_xgboost.drop(["IsHoliday", "Type"], axis = 1)
ts_xgboost.set_index('ds', inplace=True)
ts_xgboost

,Store,Dept,y,Temperature,Fuel_Price,CPI,Unemployment,Size,IsHoliday_cat
ds,,,,,,,,,
2010-02-05,1,1,24924.50,42.31,2.572,211.096358,8.106,151315,0.0
2010-02-05,1,2,50605.27,42.31,2.572,211.096358,8.106,151315,0.0
2010-02-05,1,3,13740.12,42.31,2.572,211.096358,8.106,151315,0.0
2010-02-05,1,4,39954.04,42.31,2.572,211.096358,8.106,151315,0.0
2010-02-05,1,5,32229.38,42.31,2.572,211.096358,8.106,151315,0.0
...,...,...,...,...,...,...,...,...,...
2012-10-26,1,9,41040.84,69.16,3.506,223.444251,6.573,151315,0.0
2012-10-26,1,10,26285.27,69.16,3.506,223.444251,6.573,151315,0.0
2012-10-26,1,11,22399.83,69.16,3.506,223.444251,6.573,151315,0.0


In [52]:
# Define the splitting date (change this to your desired date)
split_date = pd.to_datetime("2012-02-01")

In [53]:
# Filter data for training and testing based on the split date
X_train = ts_xgboost.loc[ts_xgboost.index < split_date].drop("y", axis=1)
y_train = ts_xgboost.loc[ts_xgboost.index < split_date]["y"]
X_test = ts_xgboost.loc[ts_xgboost.index >= split_date].drop("y", axis=1)
y_test = ts_xgboost.loc[ts_xgboost.index >= split_date]["y"]

In [54]:
# Print the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1456, 8)
y_train shape: (1456,)
X_test shape: (544, 8)
y_test shape: (544,)


In [55]:
# To tune the parameters
params = {"max_depth":6, "booster": "gbtree", "eta":0.3, "objective":"reg:linear","subsample": 0.9, "colsample_bytree": 0.7,"seed": 10}
dtrain = xgb.DMatrix(X_train, y_train)

In [56]:
dtest = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

In [57]:
#To train the model
xgboost = xgb.train(params, dtrain, 100, evals=watchlist,early_stopping_rounds= 100, verbose_eval=True)

[02:47:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:22466.39213	eval-rmse:22800.75901
[1]	train-rmse:17051.74288	eval-rmse:17337.20530
[2]	train-rmse:13473.38967	eval-rmse:13551.20440
[3]	train-rmse:11823.33878	eval-rmse:11763.41250
[4]	train-rmse:10060.29539	eval-rmse:9896.74217
[5]	train-rmse:9430.84375	eval-rmse:9741.08791
[6]	train-rmse:8018.44605	eval-rmse:8264.10552
[7]	train-rmse:7742.87707	eval-rmse:8051.72860
[8]	train-rmse:7291.36762	eval-rmse:7497.26078
[9]	train-rmse:6872.36202	eval-rmse:7052.12503
[10]	train-rmse:6772.54618	eval-rmse:7147.61270
[11]	train-rmse:6728.27350	eval-rmse:7214.89764
[12]	train-rmse:6538.95751	eval-rmse:7000.80087
[13]	train-rmse:6504.02428	eval-rmse:6969.64349
[14]	train-rmse:6362.42942	eval-rmse:6961.74730
[15]	train-rmse:6163.36425	eval-rmse:6870.48003
[1

In [58]:
#For Prediction
preds = xgboost.predict(dtest)

In [59]:
# To computer the RMSE
rms_xgboost = np.sqrt(mean_squared_error(y_test, preds))


In [61]:
rms_xgboost

7005.517290458884

In [66]:
Data = pd.DataFrame()

In [67]:
Data['Actual_value'] = y_test
Data['predicted_values'] = preds

In [68]:
Data

,Actual_value,predicted_values
ds,,
2012-02-03,23510.49,22465.404297
2012-02-03,54322.24,43274.585938
2012-02-03,12485.78,10852.996094
2012-02-03,40274.83,35228.566406
2012-02-03,28307.43,22989.445312
...,...,...
2012-10-26,41040.84,22856.908203
2012-10-26,26285.27,26093.580078
2012-10-26,22399.83,18866.777344
